# **Part 05) 파이썬으로 초보 분석가 탈출하기**

## **2_분류 모델 수행하기**

### **2.1 데이터 탐색하기**

#### **데이터 가져오기**

In [1]:
# pandas 라이브러리 가져오기
import pandas as pd

# 주어진 데이터 파일을 모두 읽어서, 각 DataFrame 변수에 저장하기
x_train = pd.read_csv('data/titanic_x_train.csv', encoding='cp949')
x_test = pd.read_csv('data/titanic_x_test.csv', encoding='cp949')
y_train = pd.read_csv('data/titanic_y_train.csv', encoding='cp949')

In [2]:
y_train = y_train.rename(columns={'癤풮assengerId': 'PassengerId'})

In [3]:
# x_train, x_test의 상위 1개 행을 확인하기
print(x_train.head(1).T)

print(x_test.head(1).T)

                                   0
PassengerId                        1
티켓등급                               3
승객이름         Braund, Mr. Owen Harris
성별                              male
나이                              22.0
형제자매배우자수                           1
부모자식수                              0
티켓번호                       A/5 21171
운임요금                            7.25
객실번호                             NaN
선착장                                S
                            0
PassengerId               892
티켓등급                        3
승객이름         Kelly, Mr. James
성별                       male
나이                       34.5
형제자매배우자수                    0
부모자식수                       0
티켓번호                   330911
운임요금                   7.8292
객실번호                      NaN
선착장                         Q


In [4]:
# y_train의 상위 5개 행을 가져오기
print(y_train.head())

   PassengerId  Survived
0            1         0
1            2         1
2            3         1
3            4         1
4            5         0


#### **행/열 확인하기**

In [5]:
# 각 데이터 세트의 행과 열의 개수를 확인하기
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)

(891, 11)
(418, 11)
(891, 2)


#### **요약 정보 확인하기**

In [6]:
# x_train 세트의 요약 정보 확인하기
print(x_train.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   티켓등급         891 non-null    int64  
 2   승객이름         891 non-null    object 
 3   성별           891 non-null    object 
 4   나이           714 non-null    float64
 5   형제자매배우자수     891 non-null    int64  
 6   부모자식수        891 non-null    int64  
 7   티켓번호         891 non-null    object 
 8   운임요금         891 non-null    float64
 9   객실번호         204 non-null    object 
 10  선착장          889 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 76.7+ KB
None


In [7]:
# '성별' 컬럼의 중복 제거한 값과 개수 확인하기
print(x_train['성별'].unique().size)
print(x_train['성별'].unique())

# '선착장' 컬럼의 중복 제거한 값과 개수 확인하기
print(x_train['선착장'].unique().size)
print(x_train['선착장'].unique())

# 나머지 3개 컬럼의 중복 제거한 값의 개수 확인하기
print(x_train['승객이름'].unique().size)
print(x_train['티켓번호'].unique().size)
print(x_train['객실번호'].unique().size)

2
['male' 'female']
4
['S' 'C' 'Q' nan]
891
681
148


#### **기초통계량 확인하기**

In [8]:
# x_train의 기초통계량을 확인하고, 가독성을 위해 행/열 바꿔서 출력하기
print(x_train.describe().T)

             count        mean         std   min       25%       50%    75%  \
PassengerId  891.0  446.000000  257.353842  1.00  223.5000  446.0000  668.5   
티켓등급         891.0    2.308642    0.836071  1.00    2.0000    3.0000    3.0   
나이           714.0   29.699118   14.526497  0.42   20.1250   28.0000   38.0   
형제자매배우자수     891.0    0.523008    1.102743  0.00    0.0000    0.0000    1.0   
부모자식수        891.0    0.381594    0.806057  0.00    0.0000    0.0000    0.0   
운임요금         891.0   32.204208   49.693429  0.00    7.9104   14.4542   31.0   

                  max  
PassengerId  891.0000  
티켓등급           3.0000  
나이            80.0000  
형제자매배우자수       8.0000  
부모자식수          6.0000  
운임요금         512.3292  


#### **독립변수와 종속변수의 관계 확인하기**

In [9]:
# x_train, y_train을 세로 방향으로 통합한 후, data 변수에 저장하기
data = pd.concat([x_train, y_train], axis=1)

In [10]:
# '성별' 컬럼에 따라 Survived의 평균값을 구하기
print(data.groupby(['성별'])['Survived'].mean())

성별
female    0.742038
male      0.188908
Name: Survived, dtype: float64


In [11]:
# '티켓등급' 컬럼에 따라 Survived의 평균값을 구하기
print(data.groupby(['티켓등급'])['Survived'].mean())

티켓등급
1    0.629630
2    0.472826
3    0.242363
Name: Survived, dtype: float64


In [12]:
# '선착장' 컬럼에 따라 Survived의 평균값을 구하기
print(data.groupby(['선착장'])['Survived'].mean())

선착장
C    0.553571
Q    0.389610
S    0.336957
Name: Survived, dtype: float64


`성별`, `티켓등급`, `선착장` 컬럼들은 최종적으로 분류해야 하는 `Survived` 컬럼과 명확한 관계가 있으므로 데이터 분석 대상에 반드시 포함

### **2.2 전처리하기**

**테스트 데이터인 x_test도** 값을 예측하는 과정에 사용하므로 <span style="color:red">**동일한 전처리 과정을 적용**</span>해야 함-!!

#### **불필요한 컬럼 삭제하기**

추후 테스트 데이터의 `PassengerId` 컬럼 값은 최종 제출하는 결과 파일에 포함되어야 하므로 `x_test_passenger_id` 변수를 별도로 저장하여 관리

In [13]:
# 테스트 데이터의 PassengerId 값은 x_test_passenger_id 변수에 저장하기
x_test_passenger_id = x_test['PassengerId']

In [14]:
# PassengerId 컬럼을 삭제하기
x_train = x_train.drop(columns=['PassengerId'])
x_test = x_test.drop(columns=['PassengerId'])
y_train = y_train.drop(columns=['PassengerId'])

# 컬럼이 삭제된 상위 5개의 행 확인하기
print(x_train.head())
print(y_train.head())

   티켓등급                                               승객이름      성별    나이  \
0     3                            Braund, Mr. Owen Harris    male  22.0   
1     1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0   
2     3                             Heikkinen, Miss. Laina  female  26.0   
3     1       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0   
4     3                           Allen, Mr. William Henry    male  35.0   

   형제자매배우자수  부모자식수              티켓번호     운임요금  객실번호 선착장  
0         1      0         A/5 21171   7.2500   NaN   S  
1         1      0          PC 17599  71.2833   C85   C  
2         0      0  STON/O2. 3101282   7.9250   NaN   S  
3         1      0            113803  53.1000  C123   S  
4         0      0            373450   8.0500   NaN   S  
   Survived
0         0
1         1
2         1
3         1
4         0


In [15]:
# 학습 데이터와 테스트 데이터에서 '티켓번호' 컬럼 삭제하기
x_train = x_train.drop(columns=['티켓번호'])
x_test = x_test.drop(columns=['티켓번호'])

In [16]:
# 학습 데이터와 테스트 데이터에서 '승객이름' 컬럼 삭제하기
x_train = x_train.drop(columns=['승객이름'])
x_test = x_test.drop(columns=['승객이름'])

#### **결측치 처리하기**

In [17]:
# '나이', '객실번호', '선착장' 컬럼의 결측값 개수 세기
print(x_train['나이'].isnull().sum())
print(x_train['객실번호'].isnull().sum())
print(x_train['선착장'].isnull().sum())

177
687
2


`나이` 컬럼과 `Survived` 컬럼(생존 여부)에 대한 상관성 확인

In [18]:
# '나이'와 'Survived' 컬럼 간의 상관관계를 확인하기
print(data[['나이', 'Survived']].corr())

                나이  Survived
나이        1.000000 -0.077221
Survived -0.077221  1.000000


상관관계가 -0.08로 매우 낮아, `나이` 컬럼은 분석 대상에서 제외

In [19]:
# x_train에서 '나이' 컬럼을 삭제하기
x_train = x_train.drop(columns=['나이'])

# x_test에서 '나이' 컬럼을 삭제하기
x_test = x_test.drop(columns=['나이'])

In [20]:
# x_train의 '객실번호' 값에서 중복을 제외한 값의 개수 세기
print(x_train['객실번호'].unique().size)

# x_train에서 '객실번호' 컬럼을 삭제하기
x_train = x_train.drop(columns=['객실번호'])

# x_test에서 '객실번호' 컬럼을 삭제하기
x_test = x_test.drop(columns=['객실번호'])

148


`선착장` 컬럼은 학습 데이터의 결측치가 2개뿐이므로, **가장 많은 비율을 차지하는 값으로 적용**

In [21]:
# '선착장' 컬럼별로 값의 개수를 세기
print(x_train.groupby(['선착장'])['선착장'].count())

선착장
C    168
Q     77
S    644
Name: 선착장, dtype: int64


In [22]:
# x_train의 '선착장' 컬럼 결측치는 'S' 값으로 적용하기
x_train['선착장'] = x_train['선착장'].fillna('S')

# x_train의 '선착장' 컬럼에 결측치가 있는지 다시 확인하기
print(x_train['선착장'].isnull().sum())

0


In [23]:
# x_test의 '선착장' 컬럼에 결측치가 있는지 다시 확인하기
print(x_test['선착장'].isnull().sum())

0


#### **범주형 변수를 인코딩하기**

In [24]:
# 학습 데이터의 '성별' 컬럼을 인코딩하기
x_train['성별'] = x_train['성별'].replace('male', 0).replace('female', 1)

# 테스트 데이터의 '성별' 컬럼을 인코딩하기
x_test['성별'] = x_test['성별'].replace('male', 0).replace('female', 1)

**원-핫 인코딩** → `pandas` 라이브러리의 ***`get_dummies()`*** 함수

In [25]:
# x_train의 '선착장' 컬럼에 대해 원-핫 인코딩을 수행한 후, '선착장_dummy'에 저장하기
선착장_dummy = pd.get_dummies(x_train['선착장'], drop_first=True).rename(columns={'Q': '선착장_Q', 'S': '선착장_S'})

# 기존 x_train의 우축에 '선착장_dummy' 변수를 덧붙여, x_train에 다시 저장하기
x_train = pd.concat([x_train, 선착장_dummy], axis=1)

# x_train의 상위 5개 행 확인하기
print(x_train.head())

   티켓등급  성별  형제자매배우자수  부모자식수     운임요금 선착장  선착장_Q  선착장_S
0     3   0         1      0   7.2500   S      0      1
1     1   1         1      0  71.2833   C      0      0
2     3   1         0      0   7.9250   S      0      1
3     1   1         1      0  53.1000   S      0      1
4     3   0         0      0   8.0500   S      0      1


In [26]:
# 학습 데이터에서 '선착장' 컬럼 삭제하기
x_train = x_train.drop(columns=['선착장'])

In [27]:
# x_test의 '선착장' 컬럼에 대해 원-핫 인코딩을 수행하고, 기존 컬럼은 삭제하기
선착장_dummy = pd.get_dummies(x_test['선착장'], drop_first=True).rename(columns={'Q': '선착장_Q', 'S': '선착장_S'})

# 기존 x_test의 우측에 '선착장_dummy' 변수를 덧붙여, x_test에 다시 저장하기
x_test = pd.concat([x_test, 선착장_dummy], axis=1)

# 테스트 데이터에서 '선착장' 컬럼 삭제하기
x_test = x_test.drop(columns=['선착장'])

#### **파생변수 만들기**

In [28]:
# '형제자매배우자수'와 '부모자식수' 컬럼을 더하여 '가족수'라는 컬럼을 만들기
x_train['가족수'] = x_train['형제자매배우자수'] + x_train['부모자식수']

# 결과 확인을 위해 상위 10개의 행 확인하기
print(x_train[['형제자매배우자수', '부모자식수', '가족수']].head(10))

   형제자매배우자수  부모자식수  가족수
0         1      0    1
1         1      0    1
2         0      0    0
3         1      0    1
4         0      0    0
5         0      0    0
6         0      0    0
7         3      1    4
8         0      2    2
9         1      0    1


In [29]:
# 학습 데이터에서 '형제자매배우자수', '부모자식수' 컬럼 삭제하기
x_train = x_train.drop(columns=['형제자매배우자수', '부모자식수'])

In [30]:
# '형제자매배우자수'와 '부모자식수' 컬럼을 더한 '가족수'라는 컬럼을 만들기
x_test['가족수'] = x_test['형제자매배우자수'] + x_test['부모자식수']

# 테스트 데이터에서 '형제자매배우자수', '부모자식수' 컬럼 삭제하기
x_test = x_test.drop(columns=['형제자매배우자수', '부모자식수'])

### **2.3 학습하고 평가하기**

평가해야 할 데이터를 추가로 만들기 위해서, **학습 데이터를 다시 학습용과 검증용으로 분리**

#### **데이터 분리하기**

In [31]:
# 데이터를 분리하기 위한 train_test_split 함수를 가져오기
from sklearn.model_selection import train_test_split

# 학습용과 검증용을 8:2로 분리하여, 각 4개의 변수에 저장하기
X_TRAIN, X_TEST, Y_TRAIN, Y_TEST = train_test_split(x_train, y_train, test_size=0.2, random_state=10)

# 분리된 데이터의 행/열 구조를 확인하기
print(X_TRAIN.shape)
print(X_TEST.shape)
print(Y_TRAIN.shape)
print(Y_TEST.shape)

(712, 6)
(179, 6)
(712, 1)
(179, 1)


#### **데이터 학습 및 하이퍼 파라미터 튜닝하기**

**일반적으로 성능이 잘 나오는** ***<span style="color:orange">XGBClassifier</span>***

In [32]:
# xgboost 라이브러리에서 제공하는 XGBClassifier 모델을 가져오기
from xgboost import XGBClassifier

In [33]:
# XGBClassifier를 수행할 첫 번째 모델을 만들고, 공부시키기
model_1 = XGBClassifier(eval_metric='error', random_state=10)
model_1.fit(X_TRAIN, Y_TRAIN)

C:\Users\khl06\anaconda3\envs\multi\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\khl06\anaconda3\envs\multi\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, eval_metric='error',
              gamma=0, gpu_id=-1, importance_type='gain',
              interaction_constraints='', learning_rate=0.300000012,
              max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=8,
              num_parallel_tree=1, random_state=10, reg_alpha=0, reg_lambda=1,
              scale_pos_weight=1, subsample=1, tree_method='exact',
              validate_parameters=1, verbosity=None)

`eval_metric` 파라미터는 분류 모델로 학습을 수행하는 경우에 작성하는 대표값

In [34]:
# XGBClassifier를 수행할 두 번째 모델을 만들고, 공부시키기
model_2 = XGBClassifier(n_estimators=100, max_depth=5, eval_metric='error', random_state=10)
model_2.fit(X_TRAIN, Y_TRAIN)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, eval_metric='error',
              gamma=0, gpu_id=-1, importance_type='gain',
              interaction_constraints='', learning_rate=0.300000012,
              max_delta_step=0, max_depth=5, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=8,
              num_parallel_tree=1, random_state=10, reg_alpha=0, reg_lambda=1,
              scale_pos_weight=1, subsample=1, tree_method='exact',
              validate_parameters=1, verbosity=None)

#### **결과 예측하기**

In [35]:
# 학습이 완료된 모델을 통해 y_test 값을 예측하기: 최종 결과 제출용
y_test_predicted_1 = pd.DataFrame(model_1.predict(x_test)).rename(columns={0: 'Survived'})

# y_test_predicted_1 값 확이하기
print(pd.DataFrame(y_test_predicted_1).head(10))

   Survived
0         0
1         0
2         0
3         0
4         1
5         0
6         0
7         1
8         1
9         0


In [36]:
print(y_test_predicted_1.head(10))

   Survived
0         0
1         0
2         0
3         0
4         1
5         0
6         0
7         1
8         1
9         0


In [37]:
# 학습이 완료된 모델을 통해 y_test 값을 예측하기: 최종 결과 제출용
y_test_predicted_2 = pd.DataFrame(model_2.predict(x_test)).rename(columns={0: 'Survived'})

# y_test_predicted_2 값 확이하기
print(pd.DataFrame(y_test_predicted_2).head(10))

   Survived
0         0
1         1
2         0
3         0
4         1
5         0
6         0
7         1
8         1
9         0


In [38]:
# 학습이 완료된 모델을 통해 Y_TEST 값을 예측하기: 평가지표 계산용
Y_TEST_PREDICTED_1 = pd.DataFrame(model_1.predict(X_TEST))

In [39]:
# 학습이 완료된 모델을 통해 Y_TEST 값을 예측하기: 평가지표 계산용
Y_TEST_PREDICTED_2 = pd.DataFrame(model_2.predict(X_TEST))

#### **모델 평가하기**

In [40]:
# sklearn 패키지의 metrics 모듈에서 roc_auc_score 함수를 가져오기
from sklearn.metrics import roc_auc_score

##### **- 1st 평가지표 결과**

In [41]:
# 1차 학습 모델의 ROC 평가지표 값을 확인하기
print(roc_auc_score(Y_TEST, Y_TEST_PREDICTED_1))

0.7731596360628619


##### **- 2nd 평가지표 결과**

In [42]:
# 2차 학습 모델의 ROC 평가지표 값을 확인하기
print(roc_auc_score(Y_TEST, Y_TEST_PREDICTED_2))

0.7983181692859112


### **2.4 결과 제출하기**

In [43]:
# x_test_passenger_id 변수와 y_test_predicted 변수를 세로 방향으로 붙이기
print(pd.concat([x_test_passenger_id, y_test_predicted_2], axis=1))

     PassengerId  Survived
0            892         0
1            893         1
2            894         0
3            895         0
4            896         1
..           ...       ...
413         1305         0
414         1306         1
415         1307         0
416         1308         0
417         1309         1

[418 rows x 2 columns]


In [44]:
# 앞의 출력 결과를 final 변수에 저장하기
final = pd.concat([x_test_passenger_id, y_test_predicted_2], axis=1)

# final 변수를 data 디렉토리 하위에 13579.csv 이름으로 저장하기
final.to_csv('data/13579.csv', index=False)

In [45]:
# 제출된 파일의 정상 여부를 확인하기
final = pd.read_csv('data/13579.csv')
print(final)

     PassengerId  Survived
0            892         0
1            893         1
2            894         0
3            895         0
4            896         1
..           ...       ...
413         1305         0
414         1306         1
415         1307         0
416         1308         0
417         1309         1

[418 rows x 2 columns]


In [46]:
### 최종 제출 코드 ###

# 데이터 가져오기
import pandas as pd
x_train = pd.read_csv('data/titanic_x_train.csv', encoding='cp949')
x_test = pd.read_csv('data/titanic_x_test.csv', encoding='cp949')
y_train = pd.read_csv('data/titanic_y_train.csv', encoding='cp949')
y_train = y_train.rename(columns={'癤풮assengerId': 'PassengerId'})

# 전처리하기
x_test_passenger_id = x_test['PassengerId']
x_train = x_train.drop(columns=['PassengerId'])
x_test = x_test.drop(columns=['PassengerId'])
y_train = y_train.drop(columns=['PassengerId'])
x_train = x_train.drop(columns=['티켓번호'])
x_test = x_test.drop(columns=['티켓번호'])
x_train = x_train.drop(columns=['승객이름'])
x_test = x_test.drop(columns=['승객이름'])
x_train = x_train.drop(columns=['나이'])
x_test = x_test.drop(columns=['나이'])
x_train = x_train.drop(columns=['객실번호'])
x_test = x_test.drop(columns=['객실번호'])

# 결측치 처리하기
x_train['선착장'] = x_train['선착장'].fillna('S')
x_test['선착장'] = x_test['선착장'].fillna('S')

# 인코딩 수행하기
x_train['성별'] = x_train['성별'].replace('male', 0).replace('female', 1)
x_test['성별'] = x_test['성별'].replace('male', 0).replace('female', 1)
선착장_dummy = pd.get_dummies(x_train['선착장'], drop_first=True).rename(columns={'Q': '선착장_Q', 'S': '선착장_S'})
x_train = pd.concat([x_train, 선착장_dummy], axis=1)
x_train = x_train.drop(columns=['선착장'])
선착장_dummy = pd.get_dummies(x_test['선착장'], drop_first=True).rename(columns={'Q': '선착장_Q', 'S': '선착장_S'})
x_test = pd.concat([x_test, 선착장_dummy], axis=1)
x_test = x_test.drop(columns=['선착장'])

# 파생변수 만들기
x_train['가족수'] = x_train['형제자매배우자수'] + x_train['부모자식수']
x_train = x_train.drop(columns=['형제자매배우자수', '부모자식수'])
x_test['가족수'] = x_test['형제자매배우자수'] + x_test['부모자식수']
x_test = x_test.drop(columns=['형제자매배우자수', '부모자식수'])

# 데이터 분리하기
from sklearn.model_selection import train_test_split
X_TRAIN, X_TEST, Y_TRAIN, Y_TEST = train_test_split(x_train, y_train, test_size=0.2, random_state=10)

# 모델 학습하기
from xgboost import XGBClassifier
model = XGBClassifier(n_estimators=100, max_depth=5, eval_metric='error', random_state=10)
model.fit(X_TRAIN, Y_TRAIN)

# 최종 결과를 파일로 저장하기
y_test_predicted = pd.DataFrame(model.predict(x_test)).rename(columns={0: 'Survived'})
final = pd.concat([x_test_passenger_id, y_test_predicted], axis=1)
final.to_csv('data/13579.csv', index=False)

# 제출된 파일의 정상 여부를 확인하기
final = pd.read_csv('data/13579.csv')
print(final)

     PassengerId  Survived
0            892         0
1            893         1
2            894         0
3            895         0
4            896         1
..           ...       ...
413         1305         0
414         1306         1
415         1307         0
416         1308         0
417         1309         1

[418 rows x 2 columns]


C:\Users\khl06\anaconda3\envs\multi\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\khl06\anaconda3\envs\multi\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
